# Installation and Suppression warping

In [ ]:
import os
import sys
import warnings
import numpy

def warn(*args, **kwargs):
    pass

warnings.warn = warn
warnings.simplefilter(action='ignore', category=FutureWarning)
numpy.seterr(divide = 'ignore') 

sys.path.append(os.path.dirname(os.path.abspath('')))

!pip install -e ..[tf,tensorforce,baselines,ccxt,fbm] -U

# Data loading

In [ ]:
import pandas as pd
from tensortrade.exchanges.simulated import SimulatedExchange

df = pd.read_csv('../data/Coinbase_BTCUSD_1h.csv', skiprows=1)
exchange = SimulatedExchange(data_frame=df, base_instrument='USD', pretransform=True)

exchange.data_frame

# Data preprocessing (normalization)

# Feature pipeline

In [ ]:
from tensortrade.features import FeaturePipeline
from tensortrade.features.scalers import MinMaxNormalizer
from tensortrade.features.stationarity import FractionalDifference

normalize_price = MinMaxNormalizer(["open", "high", "low", "close"])
difference_all = FractionalDifference(difference_order=0.6)

feature_pipeline = FeaturePipeline(steps=[normalize_price, difference_all])

exchange.feature_pipeline = feature_pipeline

exchange.data_frame

# Action scheme

In [ ]:
from tensortrade.actions import ManagedRiskOrders
from tensortrade.instruments import TradingPair
from tensortrade.instruments import Instrument
#from tensortrade.orders.criteria import 

BTC = Instrument('BTC', 8, 'Bitcoin')
USD = Instrument('USD', 2, 'U.S. Dollar')

action_scheme = ManagedRiskOrders(pairs=TradingPair(BTC, USD), trade_sizes = 10)

# Reward 

In [ ]:
from tensortrade.rewards import SimpleProfit

reward_scheme = SimpleProfit()

# Enviornment

In [ ]:
from tensortrade.environments import TradingEnvironment
from tensortrade.wallets import Portfolio
from tensortrade.wallets import Wallet
from tensortrade.instruments import Quantity

wallets = [Wallet(exchange=exchange, quantity=Quantity(instrument=USD, size=10000)), 
          Wallet(exchange=exchange, quantity=Quantity(instrument=BTC, size=0))]

portfolio = Portfolio(base_instrument='USD', wallets=wallets)

environment = TradingEnvironment(exchange=exchange,
                                 feature_pipeline=feature_pipeline,
                                 action_scheme=action_scheme,
                                 reward_scheme=reward_scheme,
                                 portfolio=portfolio)

# Agent

In [ ]:
from stable_baselines.common.policies import MlpLnLstmPolicy
from stable_baselines import PPO2

model = PPO2
policy = MlpLnLstmPolicy
params = { "learning_rate": 1e-5, 'nminibatches': 1 }

# Training strategy

In [3]:
from tensortrade.strategies import StableBaselinesTradingStrategy

strategy = StableBaselinesTradingStrategy(environment=environment,
                                          model=model,
                                          policy=policy,
                                          model_kwargs=params)

performance = strategy.run(steps=10000)

KeyboardInterrupt: 

# Visualize the result

In [ ]:
%matplotlib inline

performance.net_worth.plot()

# Save and Load the Agent

In [ ]:
strategy.save_agent(path="../agents/ppo_btc_1h")

In [ ]:
new_strategy = StableBaselinesTradingStrategy(environment=environment,
                                          model=model,
                                          policy=policy,
                                          model_kwargs=params)

new_strategy.restore_agent(path="../agents/ppo_btc_1h")

# Strategy Evaluation

In [ ]:
import pandas as pd
from tensortrade.environments import TradingEnvironment
from tensortrade.exchanges.simulated import SimulatedExchange

df = pd.read_csv('../data/Coinbase_BTCUSD_d.csv', skiprows=1)
exchange = SimulatedExchange(data_frame=df,
                             feature_pipeline=feature_pipeline,
                             base_instrument='USD',
                             pretransform=True)

environment = TradingEnvironment(exchange=exchange,
                                 action_scheme=action_scheme,
                                 reward_scheme=reward_scheme)

new_strategy.environment = environment

test_performance = new_strategy.run(steps=2000)

In [ ]:
%matplotlib inline

test_performance.net_worth.plot()